In [82]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '../SOM-LVQ')
import LVQ
from random import randrange

In [83]:
train = pd.read_csv(r"..\data\SD-2X_rocktype.csv")
x_train = train.iloc[:, 0:6].values
y_train = train.iloc[:, 6].values
y_train[y_train==4] = 0

In [84]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
x_train = minmax.fit_transform(x_train)

In [85]:
test = pd.read_csv(r"..\data\SD-3X_rocktype.csv")
x_test = test.iloc[:, 0:6].values
y_test = test.iloc[:, 6].values
y_test[y_test==4] = 0
x_test = minmax.transform(x_test)

In [86]:
# Create a random subsample from the dataset with replacement
def subsample(x, y, ratio):
    sample_x = list()
    sample_y = list()
    n_sample = round(len(x) * ratio)
    while len(sample_x) < n_sample:
        index = randrange(len(x))
        sample_x.append(x[index])
        sample_y.append(y[index])
    return [np.asarray(sample_x), np.asarray(sample_y)]

In [87]:
# Bootstrap Aggregation Algorithm
def bagging(x_train, y_train, x_test, sample_size, n_LVQ):
    trees = list()
    for i in range(n_LVQ):
        sample = subsample(x_train, y_train, sample_size)
        lvq = LVQ.LVQ(sample[0], sample[1], n_classes=4, p_vectors=[], n_neurons=25)
        y_pred = [lvq.predict(instance) for instance in x_test]
        trees.append(y_pred)
    return(np.asarray(trees))

In [88]:
# Make a prediction with a list of bagged trees
def bagging_predict(predictions, n_row, n_LVQ):
    return [np.bincount([predictions[j][i] for j in range(n_LVQ)]).argmax() for i in range(n_row)]

In [89]:
a = bagging(x_train, y_train, x_test, sample_size=1, n_LVQ=7)
y_pred = bagging_predict(a, n_row=len(a[0]), n_LVQ=7)

In [90]:
from sklearn.metrics import classification_report
print (classification_report(y_test,y_pred,target_names=['0', '1', '2', '3']))

             precision    recall  f1-score   support

          0       0.96      0.70      0.81       331
          1       0.98      0.95      0.96        95
          2       0.68      0.88      0.77        26
          3       0.66      0.92      0.77       205

avg / total       0.86      0.81      0.82       657

